In [1]:
from __future__ import absolute_import
from __future__  import division
from __future__ import print_function
import tensorflow as tf
import numpy as np
from skimage.io import imread
from skimage.transform import resize
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import pyplot
from mtcnn.mtcnn import MTCNN
from matplotlib.patches import Rectangle
from PIL import Image
from PIL import ImageEnhance
from matplotlib.patches import Circle

In [2]:
import os

In [3]:
from localfiletesting import *

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17905682091569545295
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4183621632
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11638780771546671430
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1660 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [4]:
print(tf. __version__) 

2.10.0


In [5]:
def imgenhance():
  image1 = Image.open('savedImage.jpg')
  curr_bri = ImageEnhance.Sharpness(image1)
  new_bri = 1.3
  img_brightened = curr_bri.enhance(new_bri)
  im1 = img_brightened.save("bright.jpg")

  image2 = Image.open('bright.jpg')
  curr_col = ImageEnhance.Color(image2)
  new_col = 1.5
  img_col = curr_col.enhance(new_col)
  im2 = img_col.save("finalImage.jpg")

In [6]:
def draw_faces(filename, result_list):
    # load the image
    data = pyplot.imread(filename)
    # plot each face as a subplot
    for i in range(len(result_list)):
        # get coordinates
        x1, y1, width, height = result_list[i]['box']
        x2, y2 = x1 + width, y1 + height
        # define subplot
        pyplot.subplot(1, len(result_list), i+1)
        pyplot.axis('off')
        # plot face
        pyplot.imshow(data[y1:y2, x1:x2])
    # show the plot
    pyplot.savefig("faces.png")
    pyplot.show()

In [7]:
model = Det_Model(tf,wight='fightw.hdfs')

In [9]:
from tkinter import filedialog as fd
import tkinter
root = tkinter.Tk()
root.withdraw()
V_test = fd.askopenfilename()
cap = cv2.VideoCapture(V_test)
i = 0
frames = np.zeros((30, 160, 160, 3), dtype=float)
old = []
j = 0
truecount = 0
imagesaved=0
filename = 'savedImage.jpg'
my_image = 'finalImage.jpg'
face_image = 'faces.png'
ysdatav2 = np.zeros((1, 30, 160, 160, 3), dtype=float)
ysdatav2[0][:][:] = frames
prediction = pred_fight(model,ysdatav2,acuracy=0.96)
while(True):
    ret, frame = cap.read()
  
    # describe the type of font to be used 
    font = cv2.FONT_HERSHEY_SIMPLEX
    #display the text on every frame
    text_color = (0, 255, 0) #Green
    label = prediction[0]
    if label: # Violence
        text_color = (0, 0, 255) # red
        truecount = truecount + 1
    else:# No Violence
        text_color = (0, 255, 0)
    text = "Violence: {}".format(label)
    cv2.putText(frame, text, (35, 50), font,1.25, text_color, 3)
    if i > 29:
        ysdatav2 = np.zeros((1,30,160,160, 3), dtype=float)
        ysdatav2[0][:][:] = frames
        prediction = pred_fight(model,ysdatav2,acuracy=0.96)
        if label == True:
            cv2.imshow('video', frame)
            print('Violence detected here ...')
            fourcc = cv2.VideoWriter_fourcc(*'XVID')
            vio = cv2.VideoWriter("./videos/output-"+str(j)+".avi", fourcc, 10.0, (fwidth,fheight))
            #vio = cv2.VideoWrite"./videos/output-"+str(j)+".mp4", cv2.VideoWriter_fourcc(*'mp4v'), 10, (300, 400))
            for frameinss in old:
                vio.write(frameinss)
            vio.release()
        i = 0
        j += 1
        frames = np.zeros((30, 160, 160, 3), dtype=float)
        old = []
    else:
        try:
            frm = resize(frame,(160,160,3))
            old.append(frame)
            fshape = frame.shape
            fheight = fshape[0]
            fwidth = fshape[1]
            frm = np.expand_dims(frm,axis=0)
            if(np.max(frm)>1):
                frm = frm/255
            frames[i][:] = frm
        
            i+=1
            cv2.imshow('video', frame)
        except:
            pass
            if cv2.waitKey(1):
                break
        
    if(truecount == 40):
        if(imagesaved == 0):
            if(label):
                cv2.imwrite(filename,frame)
                imagesaved = 1
        imgenhance()
        # load image from file
        pixels = cv2.imread(my_image)
        # create the detector, using default weights
        detector = MTCNN()
        # detect faces in the image
        faces = detector.detect_faces(pixels)
        # display faces on the original image
        draw_faces(my_image, faces)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()

cv2.destroyAllWindows()

1/1 [==============================] - 0s 31ms/step
Violence detected here ...
1/1 [==============================] - 0s 282ms/step


<Figure size 640x480 with 0 Axes>

1/1 [==============================] - 0s 31ms/step
Violence detected here ...
1/1 [==============================] - 0s 31ms/step
Violence detected here ...
1/1 [==============================] - 0s 51ms/step
Violence detected here ...
1/1 [==============================] - 0s 31ms/step
Violence detected here ...
1/1 [==============================] - 0s 47ms/step
Violence detected here ...
1/1 [==============================] - 0s 63ms/step
Violence detected here ...
1/1 [==============================] - 0s 47ms/step
Violence detected here ...
1/1 [==============================] - 0s 47ms/step
Violence detected here ...
1/1 [==============================] - 0s 47ms/step
Violence detected here ...
1/1 [==============================] - 0s 31ms/step
Violence detected here ...
1/1 [==============================] - 0s 47ms/step
Violence detected here ...
1/1 [==============================] - 0s 63ms/step
Violence detected here ...
1/1 [==============================] - 0s 31ms/step
